Chow tests are not a different form of test so much as a method of implementing a test that allows one to understand if:
 - Structural change has taken place over time.
 - Differences exist between subgroups within a population.
 
For example, one could test whether the effectiveness of advertising budgets have changed over time or if US and Canadian markets behave the differently.

If it turns out that two data sets behave the same, it is normally better to combine the observations to estimate one set of parameters.  If they do not, then you typically need to allow for two or more sets of relationships.  Finally, once you understand the basics of running a chow test manually, you could do a modified version where only a subset of relationships are suspected to have changed.

Chow tests involve testing multiple variables simultaneously and often require the generation of new dummy and compound variables (i.e. dummy times continuous like the ever popular Male_Ad_Budget example) to implement manually.  


# Task

In this case we will use data in the workbook *Cheat_Sheet_Chow_Test_V1_0*.  We will interpret the observation as capturing the week when the observation was recorded and we will determine whether the relationship between $y$ and $x_1$, $x_2$, and $x_3$ has changed at week 50 in the model:

$$y = B_0 + B_1 x_1 + B_2 x_2+ B_3 x_3$$

## Import the data into Pandas

The data can be found in the *Structural_Change_Data* or *Modified_Structural_Change_Data* tab in the *Cheat_Sheet_Chow_Test_V1_0*.  If you use the *Modified_ Structural_Change_Data* you will not need to use queries to do recoding as is shown in step 3 below.  You should learn to do the queries yourself, as the exam may not be as kind. 

In [ ]:
import pandas as pd
import os.path as osp

data_path = osp.join(
    osp.curdir,'Data','Cheat_Sheet_Chow_Test_V1_0.xlsx')

data = pd.read_excel(
    data_path,sheet_name='Structural_Change_Data')
data.head()

## Run the Regression
Perform a quick assessment of the results.  Does anything stand out as unusual?

In [ ]:
from statsmodels.formula.api import ols
model = ols('Y ~ X1 + X2 + X3',data).fit()
model.summary()

In [ ]:
initial_hypothesis = ('X1=0,X2=0,X3=0')
model.wald_test(initial_hypothesis)

## Write a query
Create variants of X1..X3 called C1..C3 that are 0 for obs less than or equal to 50 and their respective values for obs 51 and beyond.  You will have to create a C0 variable that is 0 for observations 1 through 50 and 1 for the rest using similar logic. To do this you can use the Numpy *where* function. You will do this for each of C1 through C3.  


In [ ]:
import numpy as np

'''
We use the where function to set values in C0 as 0 where 'Obs' is less than
or equal to 50 and 1 otherwise.
'''
data['C0'] = np.where(data['Obs'] <= 50, 0, 1)

'''
Here we accomplish the same thing for C1...C3. The for loop allows us to
bypass writing the same function 3 times. This doesn't save much work here
but would in cases where you have a large amount of parameters.
'''
for i in range (1,4):
    data['C' + str(i)] = np.where(data['Obs'] <= 50, 0, data['X' + str(i)])
data.head()

In [ ]:
#Note the transition here where Obs is 51
data[47:53]

In [ ]:
data.tail()

## Running the Regression on the Query

In [ ]:
#Running a regression using all parameters
new_model = ols('Y ~ X1 + X2 + X3 + C0 + C1 + C2 + C3',data).fit()
new_model.summary()

# Performing a Hypothesis Test

In [ ]:
#Now run the chow test which is just a specific joint hypothesis test!
hypothesis = '(C0=0,C1=0,C2=0,C3=0)'
new_model.wald_test(hypothesis)

Your output should contain the null hypothesis, the unrestricted model, and the statistics we care about.  The P-value should be very low.  
 - What does this mean?  
 - How does this compare with the individual t-test results?
